In [ ]:
using Plots, Colors, DataFrames, CSV, GLM, Statistics

cd("/Users/junwong/Dropbox/Second Year/Dingel - Trade/Assignments")

# Calibrating sector level data

In [ ]:
df = DataFrame(CSV.File("data/bilateral_trade_country/bilateral_trade_sector_country.csv"))

# Total destination sector; collapse to country-origin/dest + sector-origin level
bilateral_spending_by_sectororg = combine(groupby(df, [:country_org, :country_dest, :sector_org]), :trade_flow2014 => sum, renamecols=false);
rename!(bilateral_spending_by_sectororg, :trade_flow2014 => :trade_flow2014_sectororg)

# Total spending by sector in country n 
spending_by_sectororg = combine(groupby(bilateral_spending_by_sectororg, [:country_dest, :sector_org]), :trade_flow2014_sectororg => sum, renamecols=false);
rename!(spending_by_sectororg, :trade_flow2014_sectororg => :spending_sectororg)
bilateral_spending_by_sectororg = leftjoin(bilateral_spending_by_sectororg, spending_by_sectororg, on=[:country_dest, :sector_org])
transform!(bilateral_spending_by_sectororg, [:trade_flow2014_sectororg, :spending_sectororg] => (./) => :pi_ni_j);

df = leftjoin(df, bilateral_spending_by_sectororg, on=[:country_org, :country_dest, :sector_org]);


Compute $\gamma$

In [ ]:
# Total revenue of sector k in country n 
bilateral_revenue_by_sectororg = combine(groupby(df, [:country_org, :country_dest, :sector_org]), :trade_flow2014 => sum, renamecols=false);
rename!(bilateral_revenue_by_sectororg, :trade_flow2014 => :trade_flow2014_sectororg)

revenue_by_sectororg = combine(groupby(bilateral_revenue_by_sectororg, [:country_org, :sector_org]), :trade_flow2014_sectororg => sum, renamecols=false)
rename!(revenue_by_sectororg, [:country_org, :sector_org, :trade_flow2014_sectororg] .=> [:country, :sector_dest, :revenue_sectororg])

cross_sector_spending = combine(groupby(df, [:country_dest, :sector_dest, :sector_org]), :trade_flow2014 => sum, renamecols=false)
rename!(cross_sector_spending, [:country_dest, :trade_flow2014] .=> [:country, :spending_cross_sector])
cross_sector_spending = innerjoin(cross_sector_spending, revenue_by_sectororg, on=[:country, :sector_dest])
transform!(cross_sector_spending, [:spending_cross_sector, :revenue_sectororg] => (./) => :gamma_n_jk)

# Impute gamma
nm_cross_sector = filter(:gamma_n_jk => x -> !(ismissing(x) || isnan(x)), cross_sector_spending)
nm_cross_sector = combine(groupby(nm_cross_sector, [:sector_org, :sector_dest]), :gamma_n_jk => mean)
cross_sector_spending = leftjoin(cross_sector_spending, nm_cross_sector, on=[:sector_org, :sector_dest])
for row in eachrow(cross_sector_spending)
    row[:gamma_n_jk] = ifelse(isnan(row[:gamma_n_jk]), row[:gamma_n_jk_mean], row[:gamma_n_jk])
end

# Average gamma across countries
gamma_bar = combine(groupby(cross_sector_spending, [:sector_dest, :sector_org]), :gamma_n_jk => mean);

In [ ]:
gamma_sectdest = combine(groupby(cross_sector_spending, [:country, :sector_dest]), :gamma_n_jk => sum)
gamma_sectdest[!, :gamma_n_k] = 1 .- gamma_sectdest[!,:gamma_n_jk_sum];

avg_value_added_share = combine(groupby(gamma_sectdest, :sector_dest), :gamma_n_k => mean)
histogram(avg_value_added_share[!, :gamma_n_k_mean])





Value added in initial equilibrium 

In [ ]:
temp = leftjoin(bilateral_spending_by_sectororg, gamma_sectdest, on=[:country_dest => :country, :sector_org => :sector_dest])
transform!(temp, [:gamma_n_k, :trade_flow2014_sectororg] => (.*) => :value_added)
temp = combine(groupby(temp, :country_dest), :value_added => sum, renamecols=false)

international transfers

In [ ]:
d1 = combine(groupby(df, [:country_org, :country_dest, :sector_dest]), :trade_flow2014 => sum, renamecols=false);
X_nd_k = combine(groupby(d1, [:country_dest]), :trade_flow2014 => sum, renamecols=false);
rename!(X_nd_k, :trade_flow2014 => :xndk)
X_dn_k = combine(groupby(d1, [:country_org]), :trade_flow2014 => sum, renamecols=false);
rename!(X_dn_k, :trade_flow2014 => :xdnk)
temp1 = innerjoin(X_nd_k, X_dn_k, on = :country_dest => :country_org)
transform!(temp1, [:xndk, :xdnk] => (.-) => :transfer)
# this seems wrong? but unclear.... 
temp = innerjoin(temp, temp1, on=:country_dest )
transform!(temp, [:transfer, :value_added] => (./) => :ratio)
#plot here doesn't seem to be working for some reason 